In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/allstate-claims-severity/sample_submission.csv
/kaggle/input/allstate-claims-severity/train.csv.zip
/kaggle/input/allstate-claims-severity/sample_submission.csv.zip
/kaggle/input/allstate-claims-severity/train.csv
/kaggle/input/allstate-claims-severity/test.csv
/kaggle/input/allstate-claims-severity/test.csv.zip


In [2]:

import pandas as pd
import numpy as np
import xgboost as xgb
import datetime
import itertools
from scipy.stats import boxcox
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error
from sklearn import preprocessing
 
pd.options.mode.chained_assignment = None

In [3]:
multi_corr = [79,80,81,87,89,90,101,103,111]
two_corr = [2,3,9,10,11,12,13,23,36,57,72]
multi_cat_diff = [90,92,96,99,101,102,103,106,109,110,113,114,116]
skewed_num = [1,4,5,6,7,8,9,10,11,12,13]
cat2corr = [(29,30),(40,41),(43,45),(55,56),(8,65),(8,66),(104,106)]
two_avg1 = [1,2,3,4,5,6,7,9,10,11,12,13,14,16,23,24,25,26,27,28,36,38,40,44,50,53,57,72,73,76,79,80,81,82,87,89,90,103,111]

In [4]:

def logregobj(preds, dtrain):
    labels = dtrain.get_label()
    con = 2
    x = preds - labels
    grad = con * x / (np.abs(x) + con)
    hess = con ** 2 / (np.abs(x) + con) ** 2
    return grad, hess


In [5]:
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))
 
def encode(charcode):
    r = 0
    ln = len(str(charcode))
    for i in range(ln):
        r += (ord(str(charcode)[i]) - ord('A'))
    return r + 1


In [6]:
def prepro(train,test,cont_feature):
    joined = pd.concat((train, test)).reset_index(drop=True)
    skewed_feats = ['cont' + str(i) for i in skewed_num]
    for feats in skewed_feats:
        joined[feats] = joined[feats] + 1
        joined[feats], lam = boxcox(joined[feats])
 
    multi_diff_feats = ['cat' + str(i) for i in multi_cat_diff]
    for column in multi_diff_feats:
        set_train = set(train[column].unique())
        set_test = set(test[column].unique())
        remove_train = set_train - set_test
        remove_test = set_test - set_train
        remove = remove_train.union(remove_test)
        def filter_cat(x):
            if x in remove:
                return np.nan
            return x
        joined[column] = joined[column].apply(lambda x: filter_cat(x), 1)
 
    ss = StandardScaler()
    joined[cont_feature] = ss.fit_transform(joined[cont_feature].values)
    del train,test
    return joined


In [7]:
def feature_extract(joined,cont_feature):
    features = pd.DataFrame()
    features['id'] = joined['id']
    features['loss'] = np.log(joined['loss'] + 200)
 
    cat_sel = [n for n in joined.columns if n.startswith('cat')]
    for column in cat_sel:
        features[column] = pd.factorize(joined[column].values , sort=True)[0] + 1
 
    for column in cont_feature:
        features[column] = joined[column]
 
    features['cont_avg'] = joined[cont_feature].mean(axis = 1)
    features['cont_min'] = joined[cont_feature].min(axis = 1)
    features['cont_max'] = joined[cont_feature].max(axis = 1)
 
    for i in [20,40,73]:
        cat_feats = ['cat' + str(i) for i in range(1,i)]
        idx = 'cat_' + 'sum_' + str(i)
        features[idx + '_A'] = joined[cat_feats].apply(lambda x: sum(x == 'A'), axis = 1)
        features[idx + '_B'] = joined[cat_feats].apply(lambda x: sum(x == 'B'), axis = 1)
 
    cat2_feats = [('cat' + str(i), 'cat' + str(j)) for (i, j) in cat2corr]
    for feat1,feat2 in cat2_feats:
        feat_comb = feat1 + '_' + feat2
        features[feat_comb] = joined[feat1] + joined[feat2]
        features[feat_comb] = features[feat_comb].apply(encode)
 
    cat2avg_feats = [ 'cat' + str(i) for i in two_avg1]
    for feat1,feat2 in itertools.combinations(cat2avg_feats,2):
        feat_comb = feat1 + '_' + feat2
        features[feat_comb] = joined[feat1] + joined[feat2]
        features[feat_comb] = features[feat_comb].apply(encode)
 
    train = features[features['loss'].notnull()]
    test = features[features['loss'].isnull()]
    del features, joined
    return train, test


In [8]:

def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()


In [9]:
def feature_select(train,test):
    import operator
    params = {
        'min_child_weight': 100,
        'eta': 0.02,
        'colsample_bytree': 0.7,
        'max_depth': 12,
        'subsample': 0.7,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'objective': 'reg:linear',
        'verbose_eval': True,
        'seed': 12
    }
    rounds = 300
    y = train['loss']
    X = train.drop(['loss', 'id'], 1)
 
    xgtrain = xgb.DMatrix(X, label=y)
    bst = xgb.train(params, xgtrain, num_boost_round=rounds)
 
    feats = [x for x in train.columns if x not in ['id', 'loss']]
    print(len(feats))
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in feats:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1
    outfile.close()
 
    importance = bst.get_fscore(fmap='xgb.fmap')
    importance = sorted(importance.items(), key=operator.itemgetter(1), reverse = True)
    feats = [ a for (a,b) in importance]
    feats = feats[:450]
    print(len(feats))
    df = pd.DataFrame(importance, columns=['feature', 'fscore'])
    df['fscore'] = df['fscore'] / df['fscore'].sum()
    df.to_csv("feat_importance.csv", index = False)
 
    train1 = train[['id', 'loss'] + feats]
    test1 =  test[['id'] + feats]
    return train1, test1


In [10]:
def runXGB(train,test,index,RANDOM_STATE):
    train_index, test_index = index
    y = train['loss']
    X = train.drop(['loss', 'id'], 1)
    X_test = test.drop(['id'], 1)
    del train,test
    X_train, X_val = X.iloc[train_index], X.iloc[test_index]
    y_train, y_val = y.iloc[train_index], y.iloc[test_index]
 
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    xgval = xgb.DMatrix(X_val, label=y_val)
    xgtest = xgb.DMatrix(X_test)
    X_val = xgb.DMatrix(X_val)
 
    params = {
        'min_child_weight': 10,
        'eta': 0.01,
        'colsample_bytree': 0.7,
        'max_depth': 12,
        'subsample': 0.7,
        'alpha': 1,
        'gamma': 1,
        'silent': 1,
        'verbose_eval': True,
        'seed': RANDOM_STATE
    }
    rounds = 1000
 
    watchlist = [(xgtrain, 'train'), (xgval, 'eval')]
    model = xgb.train(params, xgtrain, rounds, watchlist, obj=logregobj, feval=evalerror,early_stopping_rounds=100)
 
    cv_score = mean_absolute_error(np.exp(model.predict(X_val)) - 200, np.exp(y_val) - 200)
    predict = np.exp(model.predict(xgtest)) - 200
    print("iteration = %d"%(model.best_iteration))
    return predict, cv_score


In [11]:
if __name__ == '__main__':
 
    Generate_or_read = 0 # 0 generate
    feat_sel = 1         # 1 select
    start_time = datetime.datetime.now()
    if Generate_or_read == 0:
        print("generate features...")
        train = pd.read_csv('../input/allstate-claims-severity/train.csv')
        test = pd.read_csv('../input/allstate-claims-severity/test.csv')
        test['loss'] = np.nan
        cont_feature = [n for n in train.columns if n.startswith('cont')]
        joined = prepro(train,test,cont_feature)
        train,test = feature_extract(joined, cont_feature)
        print(train.shape, test.shape)
        print(datetime.datetime.now() - start_time) 
        if feat_sel == 1:
            print("feature select...") 
            train,test = feature_select(train,test)
        train.to_csv("train.csv",index = False)
        test.to_csv("test.csv", index=False)
        print(train.shape, test.shape) 
        print(datetime.datetime.now() - start_time)
 
    else:
        print("read features...") 
        train = pd.read_csv(".../input/allstate-claims-severity/train.csv")
        test = pd.read_csv("../input/allstate-claims-severity/test.csv")
        print(train.shape, test.shape) 
 
    print("run model...") 
    nfolds = 10
    RANDOM_STATE = 113
    ids = test['id']
    predicts = np.zeros(ids.shape)
    kf = KFold(n_splits = nfolds, shuffle = True, random_state = RANDOM_STATE)
    for i, index in enumerate(kf.split(train)):
        print('Fold %d' % (i + 1))
        predict, cv_score = runXGB(train, test, index, RANDOM_STATE)
        print(cv_score) 
        predicts += predict
 
    print(datetime.datetime.now() - start_time) 
    predicts = predicts / nfolds
    submission = pd.DataFrame()
    submission['id'] = ids
    submission['loss'] = predicts
    submission.to_csv('submission.csv', index=False)

generate features...
(188318, 889) (125546, 889)
0:13:16.983566
feature select...
[04:29:56] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
[04:29:56] WARNING: ../src/learner.cc:516: 
Parameters: { silent, verbose_eval } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[04:52:50] WARNING: ../src/objective/regression_obj.cu:174: reg:linear is now deprecated in favor of reg:squarederror.
887
450
(188318, 452) (125546, 451)
0:37:01.338161
run model...
Fold 1
[04:53:41] WARNING: ../src/learner.cc:516: 
Parameters: { silent, verbose_eval } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip throug